In [1]:
import os

In [2]:
%pwd

'f:\\ML-Project\\research'

In [3]:
os.chdir('../')
%pwd

'f:\\ML-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path    

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_path: Path
    tragetcolunm: str

In [5]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories,save_json

[2025-01-06 15:02:21,497 : INFO : __init__ : Logger has been set up successfully!]


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
       config = self.config.model_evaluation
       params = self.params.ElasticNet
       schema = self.schema.TARGET_COLUMN

       create_directories([config.artifacts_root])
       model_evaluation_config = ModelEvaluationConfig(
            root_dir = Path(config.artifacts_root),
            test_data_path = Path(config.test_data_path),
            model_path = Path(config.model_path),
            all_params = params,
            metric_file_path = Path(config.metric_file_path),
            tragetcolunm = schema.name
         )
       
       return model_evaluation_config
    

In [7]:
import os
import pandas as pd 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import numpy as np

In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        predicted_qualities = model.predict(test_x)
        (rmse, mae, r2) = self.eval_metrics (test_y, predicted_qualities)
        # Saving metrics as local
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-06 15:02:23,087 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-01-06 15:02:23,102 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-01-06 15:02:23,105 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-01-06 15:02:23,108 : INFO : common : created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'artifacts_root'"